### https://github.com/HarmanDotpy/Named-Entity-Recognition-in-Pytorch/blob/main/scripts/train_bilstm_char_random_glove.py

In [1]:
import torch
torch.manual_seed(0)
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import matplotlib.pyplot as plt
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence, pad_sequence
from torch.utils.data import Dataset, DataLoader, TensorDataset
import io
import sklearn
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
import pickle as pickle
import warnings
from sklearn.exceptions import UndefinedMetricWarning
import seqeval
from seqeval.metrics import accuracy_score as seq_accuracy_score
from seqeval.metrics import classification_report as seq_classification_report
from seqeval.metrics import f1_score as seq_f1_score

In [2]:
## BILSTM model

class BiLSTM(nn.Module):
    def __init__(self, embedding_size, hidden_size, total_words, num_class, pretrained = False, pretrained_embed = None):
        super(BiLSTM, self).__init__()
        self.hidden_size = hidden_size
        self.wordembed = nn.Embedding.from_pretrained(pretrained_embed, freeze = False)
        self.dropout = nn.Dropout(p = 0.5)
        self.bilstm = nn.LSTM(embedding_size,hidden_size, bidirectional = True, batch_first = True)
        self.linear = nn.Linear(2*hidden_size, num_class) # 2 because forward and backward concatenate

    def forward(self, x, xlengths): 
        x = pack_padded_sequence(x, xlengths.cpu(), batch_first=True, enforce_sorted=False)
        x, _ = pad_packed_sequence(x, batch_first=True)
        word_embedding = self.wordembed(x) # x is of size(batchsize, seq_len), out is of size (batchsize, seq_len, embedding_size = 100)
        # word_embedding = self.fcembed(word_embedding)
        word_embedding = self.dropout(word_embedding) # dropout

        out, (h,c) = self.bilstm(word_embedding) #'out' has dimension(batchsize, seq_len, 2*hidden_size)
        out = self.linear(out) # now 'out' has dimension(batchsize, seq_len, num_class)
        out = out.view(-1, out.shape[2]) # shape (128*seqlen, 18)
        out = F.log_softmax(out, dim=1) # take the softmax across the dimension num_class, 'out' has dimension(batchsize, seq_len, num_class)
        return out

#### HELPER FUNCTIONS

In [18]:
# reading text file in python and making list of sentences (list of lists) and list of tags(list of lists)
def load_data(datapath, buildvocab_tags= True, vocab = None, nertags = None):
    if(buildvocab_tags == True):
        all_words = []
        all_tags = []
        with open(datapath) as f:
            lines = f.readlines()
            sent_num = 0
            for line in lines[2:]: #1: so that the first blank line isn't taken into account
                if(line == "\n"):
                    sent_num+=1
                else:
                    line_sep = line.split(sep = " ")
                    all_words.append(line_sep[0])
                    all_tags.append(line_sep[3][:-1])
                    
        words = list(set(all_words))
        tags = list(set(all_tags))

        vocab = {}
        vocab['<pad>'] = 0 # for padding input sequences
        vocab['<oov>'] = 1
        for i, word in enumerate(words):
            vocab[word] = i+2
            
        nertags = {}
        nertags['padtag'] = 0
        for i,nertag in enumerate(tags):
            nertags[nertag] = i+1

    train_sent = []
    train_tags = []
    with open(datapath) as f:
        lines = f.readlines()
        sent_num = 0
        sentence = []
        tag = []
        for line in lines[2:]: #1: so that the first blank line isn't taken into account
            if(line == "\n"):
                sent_num+=1
                train_sent.append(sentence)
                train_tags.append(tag)
                sentence = []
                tag = []
            else:
                line_sep = line.split(sep = " ")
                if(line_sep[0] in vocab.keys()):
                    sentence.append(vocab[line_sep[0]])
                else:
                    sentence.append(vocab['<oov>'])
                    
                tag.append(nertags[line_sep[3][:-1]])

    # padding the sentences at the end
    seq_maxlen = max(len(x) for x in train_sent)
    x_lengths = [len(x) for x in train_sent]
    Xtrain = []
    Ytrain = []
    for sent, tags in zip(train_sent, train_tags):
        length_toappend = seq_maxlen - len(sent)
        Xtrain.append(sent+[0]*length_toappend)
        Ytrain.append(tags+[0]*length_toappend)


    Xtrain = torch.Tensor(Xtrain)
    Ytrain = torch.Tensor(Ytrain)
    x_lengths = torch.Tensor(x_lengths)
    print(Xtrain.shape, Ytrain.shape, x_lengths.shape)
    
    return Xtrain, Ytrain, x_lengths, vocab, nertags


#### Loading data

In [19]:
traindatapath = 'data/train.txt'
devdatapath = 'data/dev.txt'
testdatapath = 'data/test.txt'


Xtrain, Ytrain, x_trainlengths, vocab, nertags = load_data(traindatapath, buildvocab_tags=True)
Xdev, Ydev, x_devlengths, _, _ = load_data(devdatapath, buildvocab_tags=False, vocab = vocab, nertags = nertags)

torch.Size([14986, 113]) torch.Size([14986, 113]) torch.Size([14986])
torch.Size([3465, 109]) torch.Size([3465, 109]) torch.Size([3465])


In [20]:
traindataset = TensorDataset(Xtrain, Ytrain, x_trainlengths)
Trainloader = DataLoader(traindataset, batch_size= 128, shuffle=True)

devdataset = TensorDataset(Xdev, Ydev, x_devlengths)
Devloader = DataLoader(devdataset, batch_size= 128, shuffle=True)

In [21]:
x_trainlengths

tensor([9., 2., 2.,  ..., 4., 2., 4.])

In [22]:
# DEFINE MY MODEL!!! 

pre_embeddings = 'glove'
Expname = 'BILSTM_glove'
rootpath = 'out/'
glove_embeddings_file = 'data/glove.6B.100d.txt'

In [23]:
if torch.cuda.is_available():  
    device = "cuda:0" 
else:  
    device = "cpu"  

In [24]:
# LOAD EMBEDDINGS
embedding_size = 100
if(pre_embeddings == "glove"):
    gloveembeddings_index = {}
    with io.open(glove_embeddings_file, encoding='utf8') as f:
        for line in f:
            values = line.split()
            word = values[0]
            coefs = np.asarray(values[1:],dtype='float32')
            gloveembeddings_index[word] = coefs

    #using vocab and Xtrain, Xvalid, get pretrained glove word embeddings
    glove_embeds = np.zeros((len(vocab), embedding_size))
    for word in vocab.keys():
        if(word in gloveembeddings_index.keys()):
            # for the pad word let theembedding be all zeros
            glove_embeds[vocab[word]] = gloveembeddings_index[word]
        else:
            glove_embeds[vocab[word]] = np.random.randn(embedding_size)
    word_embeds = torch.Tensor(glove_embeds)
    # print(glove_embeds.shape) # shape (vocablength , embedding dim)

if(pre_embeddings == "random"):
    num_words = len(vocab)
    word_embeds = torch.rand(num_words, embedding_size)

# hence we get word_embeds which we could use afterwards

In [25]:
# classes to be looked at for performance metrics
imp_classes = [nertags[tag] for tag in nertags.keys()]
imp_classes.remove(nertags['padtag'])
imp_classes.remove(nertags['O'])

In [26]:
model = BiLSTM(embedding_size = 100, hidden_size = 100, total_words = len(vocab), num_class = 18, pretrained = True, pretrained_embed = word_embeds).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr = 0.01)
# scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', patience=3) 
lossfunction = nn.CrossEntropyLoss()

In [27]:
print(model)

BiLSTM(
  (wordembed): Embedding(23626, 100)
  (dropout): Dropout(p=0.5, inplace=False)
  (bilstm): LSTM(100, 100, batch_first=True, bidirectional=True)
  (linear): Linear(in_features=200, out_features=18, bias=True)
)


In [28]:
def performance(y, ypred, nertags):
    y = y.numpy()
    ypred = ypred.numpy()
    mask = (y != nertags['padtag']) * (y != nertags['O'])
    y = y*mask
    ypred = ypred*mask
    acc = ((y==ypred)*mask).sum()/mask.sum()
    microf1 = f1_score(y, ypred, labels = imp_classes, average='micro')
    macrof1 = f1_score(y, ypred, labels = imp_classes, average='macro')

    return acc, microf1, macrof1

In [29]:
def validate(model, loader):
        with torch.no_grad():
            validloss = 0
            acc = 0
            microf1 = 0
            macrof1 = 0
            i = 0
            for step, (X, Y, xlen) in enumerate(loader):
                Y = pack_padded_sequence(Y, xlen, batch_first=True, enforce_sorted=False)
                Y, _ = pad_packed_sequence(Y, batch_first=True)
                ypred = model(X.long().to(device), xlen.to(device))#.permute(0, 2, 1)
                vloss = lossfunction(ypred.to('cpu'), Y.view(-1).type(torch.LongTensor))
                validloss+=vloss.item()
                acc_, microf1_, macrof1_ = performance(Y.view(-1), torch.argmax(ypred.to('cpu'), dim = 1), nertags)
                acc+=acc_
                microf1 += microf1_
                macrof1 += macrof1_
                i+=1

        return validloss/i, acc/i, microf1/i, macrof1/i

In [30]:
trainlosslist = []
trainacclist = [] #accuracy except pad, O
trainmicrof1list = []
trainmacrof1list = []

validlosslist = []
valacclist = []
valmicrof1list = []
valmacrof1list = []

In [31]:
# Model is ready now we have to train using cross entropy loss
num_epochs = 10

warnings.filterwarnings("ignore", category=UndefinedMetricWarning)

# validloss = []
model.train()
for epoch in range(num_epochs):
    if(epoch == 5):
        optimizer = torch.optim.Adam(model.parameters(), lr = 0.0001)
        
    totalloss, acc, microf1, macrof1 = 0, 0, 0, 0
    for step, (Xbatch ,Ybatch, xbatch_len) in enumerate(Trainloader):
        #make gradients 0
        optimizer.zero_grad()

        Ybatch = pack_padded_sequence(Ybatch, xbatch_len, batch_first=True, enforce_sorted=False)
        Ybatch, y_lengths = pad_packed_sequence(Ybatch, batch_first=True)

        #get output from model and claculate loss
        ypred = model(Xbatch.long().to(device), xbatch_len.to(device))#.permute(0, 2, 1)
        
        acc_, microf1_, macrof1_ = performance(Ybatch.view(-1), torch.argmax(ypred.to('cpu'), dim = 1), nertags)
        acc+= acc_
        microf1+=microf1_
        macrof1+=macrof1_
        if(step%20 == 0 and step !=0):
            print("training accuracy = {}, microF1 = {}, macroF1 = {}".format(acc/(step+1), microf1/(step+1), macrof1/(step+1)))
            
        loss = lossfunction(ypred.to('cpu'), Ybatch.view(-1).type(torch.LongTensor)) #Ybatch has dimension (batchsize, seqlen), ypred has dimension(batchsize, num_classes, seqlen)
        totalloss += loss.item()

        #backward and step
        loss.backward()
        nn.utils.clip_grad_norm_(model.parameters(), 5) # clip gradient to 5
        optimizer.step()
        
    trainlosslist.append(totalloss/(step+1))
    trainacclist.append(acc/(step+1))
    trainmicrof1list.append(microf1/(step+1))
    trainmacrof1list.append(macrof1/(step+1))

    # model validation loss and scheduler step for learning rate change if required
    val_loss, val_acc, val_microf1, val_macrof1  = validate(model, Devloader)
    validlosslist.append(val_loss)
    valacclist.append(val_acc)
    valmicrof1list.append(val_microf1)
    valmacrof1list.append(val_macrof1)
        
    # scheduler.step(val_loss)
    print('\nepoch = {}, training_loss = {}, validation_loss ={}, training_acc = {}, validation_acc ={}'.format(epoch, trainlosslist[-1], validlosslist[-1], trainacclist[-1], valacclist[-1]))        
    

training accuracy = 0.0043844130669047415, microF1 = 0.007660170284938238, macroF1 = 0.005465759757512896
training accuracy = 0.021424579351787893, microF1 = 0.03631135520314613, macroF1 = 0.020619012429528037
training accuracy = 0.054490372387793036, microF1 = 0.0832964324464358, macroF1 = 0.04711821458953126
training accuracy = 0.10471694354442534, microF1 = 0.14386338696334597, macroF1 = 0.08828613726428802
training accuracy = 0.15968397197629958, microF1 = 0.20377400363797873, macroF1 = 0.1367340886001963

epoch = 0, training_loss = 0.17300437251895162, validation_loss =0.08897496560322386, training_acc = 0.20143161519030622, validation_acc =0.4490270422811661
training accuracy = 0.5297113076852847, microF1 = 0.5800255513469111, macroF1 = 0.5160532834665492
training accuracy = 0.5582929090738242, microF1 = 0.6063941156259935, macroF1 = 0.5468235139129346
training accuracy = 0.5849203182444765, microF1 = 0.6312545451862273, macroF1 = 0.5697975647536416
training accuracy = 0.60367270

In [32]:
torch.save(model, 'out/word_bilstm.pt' )

In [33]:
model.eval()

BiLSTM(
  (wordembed): Embedding(23626, 100)
  (dropout): Dropout(p=0.5, inplace=False)
  (bilstm): LSTM(100, 100, batch_first=True, bidirectional=True)
  (linear): Linear(in_features=200, out_features=18, bias=True)
)

In [34]:
import os
if not os.path.exists(rootpath):
       os.mkdir(rootpath)

if not os.path.exists(rootpath+Expname):
    os.mkdir(rootpath+Expname)

In [35]:
#make id2tag
id2tag = {}
for tag in nertags.keys():
    if(tag == 'padtag'):
         id2tag[nertags[tag]] = 'O' # because we dont want the model to predict 'padtag' tags
    else:
        id2tag[nertags[tag]] = tag

In [36]:
def final_metrics(model,loader):
    y_predicted = []
    y_true = []
    with torch.no_grad():
        for step, (X, Y, xlen) in enumerate(loader):
            Y = pack_padded_sequence(Y, xlen, batch_first=True, enforce_sorted=False)
            Y, _ = pad_packed_sequence(Y, batch_first=True)
            ypred = model(X.long().to(device), xlen.to(device))#.permute(0, 2, 1)
            ypred = torch.argmax(ypred.to('cpu'), dim = 1)
            ypred = ypred.view(Y.shape[0], -1)
            y_predicted.append(ypred)
            y_true.append(Y)

    y_predicted_list = []
    y_true_list = []
    for i in range(len(y_predicted)):
        for j in range(y_predicted[i].shape[0]):
            sent_pred = []
            sent_true = []
            for x in range(y_predicted[i].shape[1]):
                sent_pred.append(id2tag[int(y_predicted[i][j, x])])
                sent_true.append(id2tag[int(y_true[i][j, x])])
            y_predicted_list.append(sent_pred)
            y_true_list.append(sent_true)
    
    return seq_f1_score(y_true_list, y_predicted_list), seq_accuracy_score(y_true_list, y_predicted_list), seq_classification_report(y_true_list, y_predicted_list, digits = 3)

    #return y_predicted, y_true

In [37]:
#Test DATASET
Xtest, Ytest, x_testlengths, _, _ = load_data(testdatapath, buildvocab_tags=False, vocab = vocab, nertags = nertags)

testdataset = TensorDataset(Xtest, Ytest, x_testlengths)
loader_test = DataLoader(testdataset, batch_size= 1, shuffle=False)
test_f1_conll, test_acc_conll, test_classif_report = final_metrics(model, loader_test)

print("PERFORMANCE ON Test DATA")
print('MicroF1 = {}'.format(test_f1_conll))
print('Accuracy = {}'.format(test_acc_conll))
print('------------Classification Report-------------')
print(test_classif_report)

torch.Size([3683, 124]) torch.Size([3683, 124]) torch.Size([3683])
PERFORMANCE ON Test DATA
MicroF1 = 0.6999191156645996
Accuracy = 0.9365263045108754
------------Classification Report-------------
              precision    recall  f1-score   support

         LOC      0.841     0.712     0.771      1668
        MISC      0.588     0.655     0.620       702
         ORG      0.746     0.604     0.668      1661
         PER      0.640     0.769     0.699      1617

   micro avg      0.711     0.689     0.700      5648
   macro avg      0.704     0.685     0.689      5648
weighted avg      0.724     0.689     0.701      5648



In [38]:
def out_predictions(model, loader, output_file):
    with open(output_file, 'w') as f:
        with torch.no_grad():
            for step, (X, Y, xlen) in enumerate(loader):
                Y = pack_padded_sequence(Y, xlen, batch_first=True, enforce_sorted=False)
                Y, _ = pad_packed_sequence(Y, batch_first=True)
                ypred = model(X.long().to(device), xlen.to(device))
                ypred = torch.argmax(ypred.to('cpu'), dim=1)
                ypred = ypred.view(Y.shape[0], -1)
                for i in range(len(ypred)):
                    for j in range(len(ypred[i])):
                        word = id2word[int(X[i, j])]
                        tag = id2tag[int(ypred[i, j])]
                        f.write(f"{word}\t{tag}\n")
                    f.write('\n')

In [41]:
# Assuming X is a tensor containing word IDs
id2word = {id: word for word, id in vocab.items()}  # Assuming vocab is a dictionary mapping words to IDs

In [42]:
out_predictions(model, loader_test, 'out/predictions.txt')

In [45]:
# Open the input and output files
with open('data/test.txt', 'r') as f_input, open('data/gold.txt', 'w') as f_output:
    # Read each line from the input file
    for line_number, line in enumerate(f_input):
        if line_number < 2:
            continue
        line = line.strip()
        if not line:
            f_output.write('\n')
            continue  
        columns = line.split()
        if len(columns) < 2:
            continue  
        word = columns[0]
        ner_tag = columns[-1]
        f_output.write(word + '\t' + ner_tag + '\n')


In [51]:
def readBIO(path):
    ents = []
    curEnts = []
    for line in open(path):
        line = line.strip()
        if line == '':
            ents.append(curEnts)
            curEnts = []
        elif line[0] == '#' and len(line.split('\t')) == 1:
            continue
        else:
            curEnts.append(line.split('\t')[1])
    return ents

def toSpans(tags):
    spans = set()
    for beg in range(len(tags)):
        if tags[beg][0] == 'B':
            end = beg
            for end in range(beg+1, len(tags)):
                if tags[end][0] != 'I':
                    break
            spans.add(str(beg) + '-' + str(end) + ':' + tags[beg][2:])
            #print(end-beg)
    return spans

def getInstanceScores(predPath, goldPath):
    goldEnts = readBIO(goldPath)
    predEnts = readBIO(predPath)
    entScores = []
    tp = 0
    fp = 0
    fn = 0
    for goldEnt, predEnt in zip(goldEnts, predEnts):
        goldSpans = toSpans(goldEnt)
        predSpans = toSpans(predEnt)
        overlap = len(goldSpans.intersection(predSpans))
        tp += overlap
        fp += len(predSpans) - overlap
        fn += len(goldSpans) - overlap
        
    prec = 0.0 if tp+fp == 0 else tp/(tp+fp)
    rec = 0.0 if tp+fn == 0 else tp/(tp+fn)
    f1 = 0.0 if prec+rec == 0.0 else 2 * (prec * rec) / (prec + rec)
    return f1

In [52]:
pred = 'out/predictions.txt'
gold = 'data/gold.txt'

In [55]:
score = getInstanceScores(pred,gold)
print('Span-F1 score - word-biLSTM ', score)

Span-F1 score - word-biLSTM  0.7361733931240658
